### Notebook for testing all models
#### (all adapted to dog dataset)
#### => currently all blocks for running tests on a single image or only calculating PSNR are commented out
#### => all not commented out blocks are needed to run a full test (which includes running all models on all the testing data and saving the images as well as calculating PSNR)
#### => important: when upscaling ratio changes all models have to be changed to new upscaling ratio and also the downscaling size
####    
####    

##### import statements

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Normalize
import math
import time

##### create low res version of all test images as input test data (only neede for dog dataset since test data is seperate kaggle folder; for human dataset train/test split is done manually in the data notebook, where low res is created for everything already)

In [24]:
def downsample_image(image_path, output_size):
    image = Image.open(image_path)

    # Downsampling using bicubic interpolation
    downscaled_image = image.resize(output_size, resample=Image.BICUBIC)

    return downscaled_image

# Folder with high resolution test images
folder_path = "/kaggle/input/animal-faces/afhq/val/dog"

# Creating folder for downscaled images
output_folder_path = "/kaggle/working/dog_low_res"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)


# Output size for downscaling
output_size = (128, 128)

# Loop over all files in the high res folder
for number, filename in enumerate(os.listdir(folder_path)):

  image_path = os.path.join(folder_path, filename)

  # Downscaling each image
  downsampled_image = downsample_image(image_path, output_size)

  # Saving downscaled image
  output_filename = f"downsampled_{filename}"
  output_path = os.path.join(output_folder_path, output_filename)
  downsampled_image.save(output_path)

'def downsample_image(image_path, output_size):\n    # Open the image\n    image = Image.open(image_path)\n\n    # Perform downsampling using bicubic interpolation\n    downscaled_image = image.resize(output_size, resample=Image.BICUBIC)\n\n    return downscaled_image\n\n# Folder path containing the images\nfolder_path = "/kaggle/input/animal-faces/afhq/val/dog"\n\n# Defining folder for downscaled images serving for input for modelling (&upscaling)\noutput_folder_path = "/kaggle/working/dog_low_res"\nif not os.path.exists(output_folder_path):\n    os.makedirs(output_folder_path)\n\n\n# Output size for downsampling\noutput_size = (128, 128)\n\n# Iterate over the files in the folder\nfor number, filename in enumerate(os.listdir(folder_path)):\n\n  # Construct the full path to the image file\n  image_path = os.path.join(folder_path, filename)\n\n  # Apply downsampling to the image\n  downsampled_image = downsample_image(image_path, output_size)\n\n  # Save the downscaled image\n  output_f

##### using same data loaders built for training process to load data for testing

In [25]:
low_res_folder = "/kaggle/working/dog_low_res"
high_res_folder = "/kaggle/input/animal-faces/afhq/val/dog"


# === creating dataset with all images ===
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_folder = low_res_folder
        self.high_res_folder = high_res_folder
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(self.low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(self.high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transformation to tensor
base_transform = transforms.Compose([
    transforms.ToTensor()
])

# dataset consists of pairs of high res / low res images
dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)

batch_size = 64 # batch size not relevant for testing
test_loader = DataLoader(dataset, batch_size=batch_size)

##### defining structure of all models (needed since saving a model only saves the weights)

In [26]:
class bicubic(nn.Module):
    def __init__(self):
        super(bicubic, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')

    def forward(self, x):
        x = self.interpolation(x)
        return x


In [27]:
# SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.interpolation(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))
        return x

In [28]:
# Load the trained model
class FSRCNN(nn.Module):
    def __init__(self, d=116, s=15, m=3):
        super(FSRCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)

        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)

        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])

        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        
        # Deconvolution for upscaling to desired size
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=4, padding=4, output_padding=3)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

In [29]:
class ESPCN(nn.Module):
    def __init__(self, upscale_factor=4, num_channels=3):
        super(ESPCN, self).__init__()
        self.upscale_factor = upscale_factor
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=5, padding=2)
        self.relu1 = nn.ReLU()
        
        # Subpixel convolution with pixle shuffle for upscaling to desired size
        self.conv2 = nn.Conv2d(64, num_channels * upscale_factor ** 2, kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pixel_shuffle(self.conv2(x))
        x = self.relu2(x)
        return x


### ==========================================
### Important: There are few cells which are commented out here, only comment them in when wanting to test the model on single image or calculate PSNR only without saving any images. If you want to test the models on all testing data with PSNR output and output images in a good folder structure, please skip the commented out cells and scroll further down :) 

#### loading a saved model (only run the cell of one modell at a time here)

In [30]:
# model = bicubic()
# model.load_state_dict(torch.load("/kaggle/input/bicubic/bicubic.pth", map_location=torch.device('cpu')))
# model.eval()

In [31]:
# model = FSRCNN()
# model.load_state_dict(torch.load("/kaggle/input/final-models/FSRCNN_18.pth", map_location=torch.device('cpu')))
# model.eval()

In [32]:
# model = SRCNN()
# model.load_state_dict(torch.load("/kaggle/input/final-models/SRCNN_13.pth", map_location=torch.device('cpu')))
# model.eval()

In [33]:
# model = ESPCN()
# model.load_state_dict(torch.load("/kaggle/input/final-models/ESPCN_13.pth", map_location=torch.device('cpu')))
# model.eval()

##### test the model on a single defined image

In [34]:
# # Preprocess the test image
# path = "/kaggle/working/dog_low_res"
# filename = "" # write a valid filename here
# test_image_path = path+filename
# test_image = Image.open(test_image_path)
# transform = transforms.Compose([
#     transforms.ToTensor()
# ])
# test_image = transform(test_image).unsqueeze(0)

# # Apply the model to the test image
# with torch.no_grad():
#     output_image = model(test_image)

# # Convert the output tensor to an image
# output_image = output_image.squeeze(0)
# output_image = transforms.ToPILImage()(output_image)

# # Save the output image
# output_image.save(f"./data/Human/Up/upscaled_{filename}")


##### calculate testing PSNR only (no images will be saved)

In [35]:
# import time
# test_loss = 0
# number_batches = 0
# criterion = nn.MSELoss()
# device = torch.device("cpu")

# start_time = time.time()
# for input_data, desired_data in test_loader:
#     with torch.no_grad():
#         number_batches += 1
#         print(number_batches)

#         input_data = input_data.to(device)
#         desired_data = desired_data.to(device)

#         # Forward pass
#         output_images = model(input_data)

#         # Calculate loss
#         loss = criterion(output_images, desired_data)
#         test_loss += loss.item()

# end_time = time.time()  

# elapsed_time = end_time - start_time  
# print(f"Duration (total): {elapsed_time} seconds, duration per file: {elapsed_time/len(dataset)} seconds")

# test_loss_avg = test_loss / number_batches

# psnr = 10 * math.log10(1 / test_loss_avg)

# print(f"Loss: {test_loss_avg:.4f}, PSNR: {psnr}")

### ==========================================


### apply all models to all the tesing data (saves upscaled images and calculates PSNR)
##### down below first a folder structure is created, then each model has two cells - one for loading the model and one for applying it to all test data (therefore the application cells are similar for each model, advantage is that you don't have to change anything when running the cells and can test all models at once by running all cells below)
#####

#### creating folder structure for output

In [ ]:
number_batches = 0
device = torch.device("cpu")

img_comparison_folder = "/kaggle/working/img_comparison"

if not os.path.exists(img_comparison_folder):
    os.makedirs(img_comparison_folder)

start_time = time.time() # start timer
for input_data, desired_data in test_loader:
    with torch.no_grad():
        number_batches += 1

        # creating a folder for the current batch (decision was made to split up file structure into batch folders)
        if not os.path.exists(f"/kaggle/working/img_comparison/batch_{number_batches}"):
            os.makedirs(f"/kaggle/working/img_comparison/batch_{number_batches}")

        # input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)
            
        # saving low res images from the same batch into the current batch folder
        for number, image in enumerate(input_data):
            image = image.squeeze(0)
            image = transforms.ToPILImage()(image)
            image.save(f"/kaggle/working/img_comparison/batch_{number_batches}/{number}_input.jpg")
            
        # saving original images from the same batch into the same folder
        for number, image in enumerate(desired_data):
            image = image.squeeze(0)
            image = transforms.ToPILImage()(image)
            image.save(f"/kaggle/working/img_comparison/batch_{number_batches}/{number}_original.jpg")

end_time = time.time()  # stop timer

elapsed_time = end_time - start_time  # calculate time
print(f"Time needed: {elapsed_time} seconds")


'number_batches = 0\ndevice = torch.device("cpu")\n\nimg_comparison_folder = "./data/Human/img_comparison"\n\nif not os.path.exists(img_comparison_folder):\n    os.makedirs(img_comparison_folder)\n\nstart_time = time.time()\nfor input_data, desired_data in test_loader:\n    with torch.no_grad():\n        number_batches += 1\n        if not os.path.exists(f"./data/Human/img_comparison/batch_{number_batches}"):\n            os.makedirs(f"./data/Human/img_comparison/batch_{number_batches}")\n\n        # Move input and desired images to device\n        input_data = input_data.to(device)\n        desired_data = desired_data.to(device)\n            \n        # saving low res images from the same batch into the same folder\n        for number, image in enumerate(input_data):\n            image = image.squeeze(0)\n            image = transforms.ToPILImage()(image)\n            image.save(f"./data/Human/img_comparison/batch_{number_batches}/{number}_input.jpg")\n            \n        # saving o

### bicubic

In [37]:
model = bicubic()
model.load_state_dict(torch.load("/kaggle/input/final-models/bicubic.pth", map_location=torch.device('cpu')))
model.eval()

bicubic(
  (interpolation): Upsample(scale_factor=3.0, mode='bicubic')
)

In [38]:
test_loss = 0
number_batches = 0
criterion = nn.MSELoss()
device = torch.device("cpu")

img_comparison_folder = "/kaggle/working/img_comparison"

if not os.path.exists(img_comparison_folder):
    os.makedirs(img_comparison_folder)
start_time = time.time() # start timer
for input_data, desired_data in test_loader:
    with torch.no_grad():
        number_batches += 1

        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)
        test_loss += loss.item()
        
        # saving the images
        for number, output_image in enumerate(output_images):
            output_image = output_image.squeeze(0)
            output_image = transforms.ToPILImage()(output_image)
            output_image.save(f"/kaggle/working/img_comparison/batch_{number_batches}/{number}_upscaled_bicubic.jpg")
            
        
end_time = time.time()  # stop timer

elapsed_time = end_time - start_time  # calculate time
print(f"Needed time: {elapsed_time} seconds")
test_loss_avg = test_loss / number_batches

# Print test loss
psnr = 10 * math.log10(1 / test_loss_avg)

print(f"Loss: {test_loss_avg:.4f}, PSNR: {psnr}")

Verstrichene Zeit: 358.05539178848267 Sekunden
Loss: 0.0011, PSNR: 29.57149595113906


### SRCNN

In [39]:
model = SRCNN()
model.load_state_dict(torch.load("/kaggle/input/final-models/SRCNN_13.pth", map_location=torch.device('cpu')))
model.eval()

SRCNN(
  (interpolation): Upsample(scale_factor=3.0, mode='bicubic')
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (relu1): ReLU()
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu3): ReLU()
)

In [42]:
test_loss = 0
number_batches = 0
criterion = nn.MSELoss()
device = torch.device("cpu")

img_comparison_folder = "/kaggle/working/img_comparison"

if not os.path.exists(img_comparison_folder):
    os.makedirs(img_comparison_folder)
start_time = time.time() # start timer
for input_data, desired_data in test_loader:
    with torch.no_grad():
        number_batches += 1

        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)
        test_loss += loss.item()
        
        # saving the images
        for number, output_image in enumerate(output_images):
            output_image = output_image.squeeze(0)
            output_image = transforms.ToPILImage()(output_image)
            output_image.save(f"/kaggle/working/img_comparison/batch_{number_batches}/{number}_upscaled_SRCNN.jpg")
            
        
end_time = time.time()  # stop timer

elapsed_time = end_time - start_time  # calculate time
print(f"Needed time: {elapsed_time} seconds")
test_loss_avg = test_loss / number_batches

# Print test loss
psnr = 10 * math.log10(1 / test_loss_avg)

print(f"Loss: {test_loss_avg:.4f}, PSNR: {psnr}")

### FSRCNN

In [ ]:
model = FSRCNN()
model.load_state_dict(torch.load("/kaggle/input/final-models/FSRCNN_18.pth", map_location=torch.device('cpu')))
model.eval()

In [ ]:
test_loss = 0
number_batches = 0
criterion = nn.MSELoss()
device = torch.device("cpu")

img_comparison_folder = "/kaggle/working/img_comparison"

if not os.path.exists(img_comparison_folder):
    os.makedirs(img_comparison_folder)
start_time = time.time() # start timer
for input_data, desired_data in test_loader:
    with torch.no_grad():
        number_batches += 1

        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)
        test_loss += loss.item()
        
        # saving the images
        for number, output_image in enumerate(output_images):
            output_image = output_image.squeeze(0)
            output_image = transforms.ToPILImage()(output_image)
            output_image.save(f"/kaggle/working/img_comparison/batch_{number_batches}/{number}_upscaled_FSRCNN.jpg")
            
        
end_time = time.time()  # stop timer

elapsed_time = end_time - start_time  # calculate time
print(f"Needed time: {elapsed_time} seconds")
test_loss_avg = test_loss / number_batches

# Print test loss
psnr = 10 * math.log10(1 / test_loss_avg)

print(f"Loss: {test_loss_avg:.4f}, PSNR: {psnr}")

### ESPCN

In [ ]:
model = ESPCN()
model.load_state_dict(torch.load("/kaggle/input/final-models/ESPCN_13.pth", map_location=torch.device('cpu')))
model.eval()

In [ ]:
test_loss = 0
number_batches = 0
criterion = nn.MSELoss()
device = torch.device("cpu")

img_comparison_folder = "/kaggle/working/img_comparison"

if not os.path.exists(img_comparison_folder):
    os.makedirs(img_comparison_folder)
start_time = time.time() # start timer
for input_data, desired_data in test_loader:
    with torch.no_grad():
        number_batches += 1

        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)
        test_loss += loss.item()
        
        # saving the images
        for number, output_image in enumerate(output_images):
            output_image = output_image.squeeze(0)
            output_image = transforms.ToPILImage()(output_image)
            output_image.save(f"/kaggle/working/img_comparison/batch_{number_batches}/{number}_upscaled_ESPCN.jpg")
            
        
end_time = time.time()  # stop timer

elapsed_time = end_time - start_time  # calculate time
print(f"Needed time: {elapsed_time} seconds")
test_loss_avg = test_loss / number_batches

# Print test loss
psnr = 10 * math.log10(1 / test_loss_avg)

print(f"Loss: {test_loss_avg:.4f}, PSNR: {psnr}")